<a href="https://colab.research.google.com/github/HedersonSantos/beam/blob/main/cap1/Arq1_ReadTransform_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install apache-beam[interactive]

     |████████████████████████████████| 10.1 MB 9.6 MB/s 
     |████████████████████████████████| 151 kB 44.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 255 kB 66.9 MB/s 
     |████████████████████████████████| 46 kB 3.0 MB/s 
     |████████████████████████████████| 2.3 MB 41.4 MB/s 
     |████████████████████████████████| 508 kB 53.5 MB/s 
     |████████████████████████████████| 112 kB 42.2 MB/s 
     |████████████████████████████████| 121 kB 56.8 MB/s 
     |████████████████████████████████| 793 kB 44.3 MB/s 
     |████████████████████████████████| 380 kB 52.4 MB/s 
     |████████████████████████████████| 1.1 MB 41.6 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=323a786acbcf9d317c6bc586064db6d9cea97192515b605ca7f762898e07591d
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for timeloop: filename=timeloop-1.0.2

In [3]:
!wget https://raw.githubusercontent.com/HedersonSantos/RegDatChallg/main/data/train.csv

--2022-03-29 10:48:48--  https://raw.githubusercontent.com/HedersonSantos/RegDatChallg/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1397869 (1.3M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.33M  --.-KB/s    in 0.03s   

2022-03-29 10:48:48 (38.1 MB/s) - ‘train.csv’ saved [1397869/1397869]



In [6]:
#criando um registro
import apache_beam as beam
p1 = beam.Pipeline()
p1 | "Tupla" >> beam.Create([("Cassio",32), ("Vick",21)])| "imprime Tupla" >> beam.Map(print)
p1 | "Lista" >> beam.Create([1,2,3]) | "imprime Lista" >> beam.Map(print, end=' ')
p1.run()


('Cassio', 32)
('Vick', 21)
1 2 3 

In [7]:
import apache_beam as beam
p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
registros = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("train.csv", validate=True, skip_header_lines=1)
    |"Separar por Vírgulas" >> beam.Map(lambda record: record.split(','))
    #|"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    |"Filtrar score" >> beam.Filter(lambda record: (record[1]>'10') & (record[1]<'20'))
    |"Gravar Resultados" >> beam.io.WriteToText("teste.csv")
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

In [8]:
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/dataset/valid.csv

--2022-03-29 10:49:21--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/dataset/valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 766613 (749K) [text/plain]
Saving to: ‘valid.csv’

valid.csv           100%[===================>] 748.65K  --.-KB/s    in 0.03s   

2022-03-29 10:49:21 (23.2 MB/s) - ‘valid.csv’ saved [766613/766613]



In [9]:
#FlatMap
import apache_beam as beam

palavras = ['cachorro', 'animal','Europa']

def encontrarPalavras(i):
  if i in palavras:
    return True

p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
registros = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("valid.csv", validate=True, skip_header_lines=1)
    |"Quebrar por palavras" >> beam.FlatMap(lambda record: record.split(' ')) #FlatMap irá gerar uma linha a cada espaço, i.e, uma palavra em cada linha.
    |"Encontrar palavras" >> beam.Filter(encontrarPalavras)
    |"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    |"Gravar Resultados" >> beam.io.WriteToText("teste_palavras.csv")
    
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

Europa
Europa
Europa
Europa
cachorro
Europa
animal


In [11]:
p = beam.Pipeline()

#tuplas
negros = ('Machado Assis', 'Edson Arantes', 'Barack Obama')
brancos = ('FHC', 'Carlos Drummond', 'Autelina')
indios = ('Juruna', 'Raoni', 'Galdino')
#p_collections
negros_pc = p | "Criando Pcollection negros" >> beam.Create(negros)
brancos_pc = p | "Criando Pcollection brancos" >> beam.Create(brancos)
indios_pc = p | "Criando Pcollections indios" >> beam.Create(indios)

pessoas = ((negros_pc, brancos_pc, indios_pc) | beam.Flatten()) | beam.Map(print)
p.run()



FHC
Carlos Drummond
Autelina
Juruna
Raoni
Galdino
Machado Assis
Edson Arantes
Barack Obama


In [18]:
#agregação - soma de valores
p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
Tempo_Atrasos = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("voos_sample.csv", validate=True, skip_header_lines=1)
    |"Separar voos por Vírgulas" >> beam.Map(lambda record: record.split(','))
    |"Filtrar voos atrasados" >> beam.Filter(lambda record: int(record[8])>0 ) #nesta coluna nros positivos significa voo atrasado, i.e, o tempo de atraso
    |"Criar par" >> beam.Map(lambda record: (record[4],int(record[8]))) #record 4 é a sigla do aeroporto
    |"Somar por key" >> beam.CombinePerKey(sum)
    |"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

('LAX', 92)
('HNL', 15)
('DFW', 95)
('OGG', 138)
('JFK', 220)


In [16]:
#agregação - contagem de registros
p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
Qtd_Atrasos = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("voos_sample.csv", validate=True, skip_header_lines=1)
    |"Separar voos por Vírgulas" >> beam.Map(lambda record: record.split(','))
    |"Filtrar voos atrasados" >> beam.Filter(lambda record: int(record[8])>0 ) #nesta coluna nros positivos significa voo atrasado, i.e, o tempo de atraso
    |"Criar par" >> beam.Map(lambda record: (record[4],int(record[8]))) #record 4 é a sigla do aeroporto
    |"Contar por key" >> beam.combiners.Count.PerKey()
    |"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    
    
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

('LAX', 4)
('HNL', 1)
('DFW', 1)
('OGG', 1)
('JFK', 4)


In [25]:
#CoGroupByKey Faz o join de duas pcollections.
p1 = beam.Pipeline()

Tempo_Atrasos = (
  p1
  | "Importar Dados Atraso" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines = 1)
  | "Separar por Vírgulas Atraso" >> beam.Map(lambda record: record.split(','))
  | "Pegar voos com atraso" >> beam.Filter(lambda record: int(record[8]) > 0 )
  | "Criar par atraso" >> beam.Map(lambda record: (record[4],int(record[8])))
  | "Somar por key" >> beam.CombinePerKey(sum)
#  | "Mostrar Resultados" >> beam.Map(print)
)

Qtd_Atrasos = (
  p1
  | "Importar Dados" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines = 1)
  | "Separar por Vírgulas" >> beam.Map(lambda record: record.split(','))
  | "Pegar voos com atraso qtd" >> beam.Filter(lambda record: int(record[8]) > 0 )
  | "Criar par qtd" >> beam.Map(lambda record: (record[4],int(record[8])))
  | "Contar por key" >> beam.combiners.Count.PerKey()
#  | "Mostrar Resultados QTD" >> beam.Map(print)
)

tabela_atrasos = (
    {'Qtd_Atrasos':Qtd_Atrasos,'Tempo_Atrasos':Tempo_Atrasos} 
    | "Group By" >> beam.CoGroupByKey()
    | beam.Map(print)
)

p1.run()

('LAX', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [92]})
('HNL', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [15]})
('DFW', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [95]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [138]})
('JFK', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [220]})


In [26]:
#função ParDO -> permite criar uma função de usuário e chamar no Map
p1 = beam.Pipeline()

class filtro(beam.DoFn):
  def process(self, record):
    if int(record[8])>0:
      return [record]

Tempo_Atrasos = (
  p1
  | "Importar Dados Atraso" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines = 1)
  | "Separar por Vírgulas Atraso" >> beam.Map(lambda record: record.split(','))
  | "Pegar voos com atraso" >> beam.ParDo(filtro())
  | "Criar par atraso" >> beam.Map(lambda record: (record[4],int(record[8])))
  | "Somar por key" >> beam.CombinePerKey(sum)
#  | "Mostrar Resultados" >> beam.Map(print)
)

Qtd_Atrasos = (
  p1
  | "Importar Dados" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines = 1)
  | "Separar por Vírgulas" >> beam.Map(lambda record: record.split(','))
  | "Pegar voos com atraso qtd" >> beam.Filter(lambda record: int(record[8]) > 0 )
  | "Criar par qtd" >> beam.Map(lambda record: (record[4],int(record[8])))
  | "Contar por key" >> beam.combiners.Count.PerKey()
#  | "Mostrar Resultados QTD" >> beam.Map(print)
)

tabela_atrasos = (
    {'Qtd_Atrasos':Qtd_Atrasos,'Tempo_Atrasos':Tempo_Atrasos} 
    | "Group By" >> beam.CoGroupByKey()
    | beam.Map(print)
)

p1.run()

('LAX', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [92]})
('HNL', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [15]})
('DFW', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [95]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [138]})
('JFK', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [220]})
